In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import ee
import os
import pandas as pd
import datetime
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DbLh_D4D68drQ3zMDaM4B9OgRRWb8ZDnrlG8e_lu5JE&tc=Yp_KMURJz8SBQm8W0EXzX6Xy1ZrZxNIKKwU1pbOZ7X8&cc=xE-Nht3EoCRsmVWu7SJc18fAL_vtm56SD4K_mUnDCIs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqso7iDKgvFwc2qvgPUM11m5T4UAq4WC8-jo6BXvy5Z6cVNT5GYQmU

Successfully saved authorization token.


In [10]:
# Import the Land Surface Temperature
dict_product={'Landsat9':'LANDSAT/LC09/C02/T1_L2'}
dict_product={'Landsat8':'LANDSAT/LC08/C02/T1_L2'}
# Final date of interest (exclusive).
i_date='2021-10-31'
i_date='2013-03-18'
f_date=str(datetime.date.today().strftime("%Y-%m-%d"))
banda='ST_B10'
scale=30
# Define the urban location of interest as a point near Lyon, France.

df_est=pd.read_excel(os.path.join('.','metadata_estaciones_glaciares.xlsx'))
features=[]
for index, row in df_est.iterrows():
#     print(dict(row))
#     construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['Lon'], row['Lat']])
#     print(poi_geometry)
#     construct the attributes (properties) for each point 
    poi_properties = dict(row)
#     construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
#     print(poi_feature)
    features.append(poi_feature)

# final Feature collection assembly
ee_fc = ee.FeatureCollection(features)

In [ ]:
def rasterExtraction(image):
    feature = image.sampleRegions(collection=ee_fc,scale=scale)
    return feature

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def getCelsius(image):
    
    # LST
    lst = image.select(banda).multiply(0.00341802).add(149.0).add(-273.15)\
    .rename("LST")
    image = image.addBands(lst)

    return(image)

landsat_data = ee.ImageCollection(dict_product.get(list(dict_product.keys())[0])) \
    .filterDate(i_date,f_date).map(getCelsius).map(addDate)
  
results=landsat_data.filterBounds(ee_fc).select('LST').map(addDate)\
.map(rasterExtraction).flatten()
sample_result=results.first().getInfo()
#extract the properties column from feature collection
#column order may not be as out sample data order

columns=list(sample_result['properties'].keys())
#order data columns as per sample data order
#You can modify this for better optimizaction
column_df=list(df_est.columns)
column_df.extend(['LST','date'])
nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)),
                                    column_df).values().get(0)
data = nested_list.getInfo()
df = pd.DataFrame(data, columns=column_df)
df.index=pd.to_datetime(df['date'],format="%Y%m%d")
df=df[[x for x in df.columns if x!='date']]
df_pivot=pd.pivot_table(df,values='LST',index=df.index,columns=df['COD_BNA'])
print(df_pivot.head())
df_pivot.to_excel(os.path.join('.','LST_'+list(dict_product.keys())[0]+'.xlsx'))